In [78]:
%load_ext rpy2.ipython
%load_ext autoreload
%autoreload 2
#%matplotlib qt

In [3]:
from skdim import global_id
from skdim import local_id
import skdim
import multiprocessing as mp
import scipy
import numpy as np
import pandas as pd
import rpy2
import rpy2.robjects as ro
import rpy2.robjects.numpy2ri
import rpy2.robjects.packages as rpackages

from sklearn.neighbors import NearestNeighbors
import sklearn.datasets
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

rpy2.robjects.numpy2ri.activate()
utils = rpackages.importr('utils')
#utils.install_packages('intrinsicDimension')
#utils.install_packages('ider')
intdimr = rpackages.importr('intrinsicDimension')
ider   = rpackages.importr('ider')
#r_base = rpackages.importr('base')

In [4]:
def compute_betas(max_dim):
    #COMPUTE_BETAS Computes betas between one and max_dim. 
    #   betas = compute_betas( max_dim ) Computes the variance of the angle between two Gaussian
    #   vectors in RR^d, it does so for each d between 1 and max_dim.

    betas = np.zeros(max_dim-2)
    betas[:2] = np.array([np.pi**2/12, np.pi**2/4 - 2])

    if (max_dim >= 4):
        for ii in range(4,max_dim):
            betas[ii-1] = betas[ii-3] - 2/(ii-1)**2

    betas = np.concatenate((np.array([pi**2/4]), betas))

    return betas[:,None]

def compute_cuts(max_dim):
    #COMPUTE_CUTS Computes the cuts that define the thresholds to decide between
    #   etas = compute_cuts(max_dim) Computes the cuts to decide between any
    #   dimension between 1 and max_dim.
    betas = compute_betas(max_dim)
    etas = np.zeros(len(betas)-1,1)
    for ii in range(len(etas)):
        etas[ii] = (betas[ii] + betas[ii+1])/2
    
    return etas

In [13]:
data = np.zeros((30,10))
data[:,:5] = skdim.gendata.hyperBall(n_points = 30, n_dim = 5, radius = 1, random_state = 0)

In [92]:
assert np.allclose(
    (skdim.global_id.MLE().fit(data).dimension_,
    skdim.global_id.MLE(k=5).fit(data).dimension_,
    skdim.global_id.MLE(n = 20, sigma=.1, dnoise='dnoiseGaussH').fit(data).dimension_,
    skdim.global_id.MLE(unbiased=True).fit(data).dimension_,
    skdim.global_id.MLE(K=10, neighborhood_based=False).fit(data).dimension_,
    skdim.global_id.MLE(neighborhood_aggregation='mean').fit(data).dimension_,
    ),
    (intdimr.maxLikGlobalDimEst(data,k=20)[0][0],
    intdimr.maxLikGlobalDimEst(data,k=5)[0][0],
    intdimr.maxLikGlobalDimEst(data,k=20,n = 20, sigma=.1, dnoise='dnoiseGaussH')[0][0],
    intdimr.maxLikGlobalDimEst(data,k=20,unbiased=True)[0][0],
    intdimr.maxLikGlobalDimEst(data,k=20,K=10, neighborhood_based=False)[0][0],
    intdimr.maxLikGlobalDimEst(data,k=20,neighborhood_aggregation='mean')[0][0]
    )

array([2.73873109, 2.1619804 , 2.29577097, 3.13300883, 2.68340134,
       2.7744833 , 2.16936429, 2.63976628, 3.73803225, 2.68782875,
       3.3468158 , 3.3375878 , 2.58747076, 3.49101585, 2.84130906,
       3.68365681, 3.5694458 , 3.50015857, 2.78155042, 4.47995493,
       4.0897491 , 3.79781342, 2.62101409, 3.03665971, 2.84263582,
       3.82414291, 2.65283788, 3.46869821, 3.851816  , 3.13222988])